In [1]:
import xarray as xr
# import gcsfs
# import fsspec
import cdsapi
import time
import numpy as np

In [2]:
fs = gcsfs.GCSFileSystem(project='handy-cathode-260804', token='/home/jovyan/gs-transferrer-cred.json')
gcsmap = gcsfs.mapping.GCSMap('em-gs-1/era5T', gcs=fs, check=True, create=False)

gcsmap

NameError: name 'gcsfs' is not defined

In [4]:
from src.utils import keys
key = keys()['COPERNICUS']['key']
uri = keys()['COPERNICUS']['uri']

In [14]:
c = cdsapi.Client(url='https://cds.climate.copernicus.eu/api/v2', verify=0, quiet=True, key='%s:%s'%(uri, key))

In [15]:
variables = ['mean_total_precipitation_rate']

years = ['2020']

months = ['09' 
          ]

days = [str(x) for x in np.arange(1,32).tolist()]

times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', 
         '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', 
         '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', 
         '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

In [17]:
res = c.retrieve("reanalysis-era5-single-levels",
  {
    'product_type': 'reanalysis',
    'variable': variables,
    'year': years,
    'month': months,
    'day':  days,
    'time': times,
    'format': 'netcdf',
    'area'          : [38, 68, 40, 70],
    'grid'          : [0.25, 0.25]
  },
  
                )

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2021-08-12 09:46:04,679 INFO Welcome to the CDS
2021-08-12 09:46:04,680 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certific

In [18]:
res.location.split('/')[-1]

'adaptor.mars.internal-1628703871.50876-23032-14-564e7ebf-0a24-4a56-801b-e42ab8340849.nc'

In [103]:
def upload_ERA5_to_GCS(path, variables, years, months, days, times):

    for year in years:
        for month in months:
            print([year, month])
            c = cdsapi.Client()

            res = c.retrieve("reanalysis-era5-single-levels",
              {
                'product_type': 'reanalysis',
                'variable': variables,
                'year': year,
                'month': month,
                'day':  days,
                'time': times,
                'format': 'netcdf',
                'area'          : [38, 68, 8, 98],
                'grid'          : [0.25, 0.25]
              },
              path
                            )
            print(res)
            
            time.sleep(30)
            
            data = xr.load_dataset('/home/jovyan/to_upload_to_gcs/temp.nc')
            data.to_zarr(store=gcsmap, mode='a', append_dim='time')
            
            print('Uploaded to GCS.')

In [114]:
path = "/home/jovyan/to_upload_to_gcs/temp.nc"

variables = ['100m_u_component_of_wind', '100m_v_component_of_wind', '10m_u_component_of_neutral_wind',
            '10m_v_component_of_neutral_wind', '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'boundary_layer_height', 'convective_inhibition', 'mean_total_precipitation_rate']

years = ['2020']

months = ['09' 
          ]

days = [str(x) for x in np.arange(1,32).tolist()]

times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', 
         '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', 
         '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', 
         '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

In [105]:
%%time 
upload_ERA5_to_GCS(path, variables, years, months, days, times)

['2020', '09']


2020-11-04 11:19:02,395 INFO Welcome to the CDS
2020-11-04 11:19:02,396 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-11-04 11:19:02,576 INFO Request is queued
2020-11-04 11:19:03,681 INFO Request is running
2020-11-04 11:25:22,675 INFO Request is completed
2020-11-04 11:25:22,676 INFO Downloading http://136.156.133.42/cache-compute-0014/cache/data0/adaptor.mars.internal-1604488743.0133224-633-24-edf3d929-55b8-4352-a8ce-3ac37abbdd43.nc to /home/jovyan/to_upload_to_gcs/temp.nc (402.1M)
2020-11-04 11:25:55,557 INFO Download rate 12.2M/s 


Result(content_length=421668124,content_type=application/x-netcdf,location=http://136.156.133.42/cache-compute-0014/cache/data0/adaptor.mars.internal-1604488743.0133224-633-24-edf3d929-55b8-4352-a8ce-3ac37abbdd43.nc)


ValueError: append_dim was set along with mode='w', either set mode='a' or don't set it.

In [107]:
data = xr.load_dataset('/home/jovyan/to_upload_to_gcs/temp.nc')
# data.to_zarr(store=gcsmap, mode='a', append_dim='time')
data

<xarray.Dataset>
Dimensions:    (expver: 2, latitude: 121, longitude: 121, time: 720)
Coordinates:
  * longitude  (longitude) float32 68.0 68.25 68.5 68.75 ... 97.5 97.75 98.0
  * latitude   (latitude) float32 38.0 37.75 37.5 37.25 ... 8.75 8.5 8.25 8.0
  * expver     (expver) int32 5 1
  * time       (time) datetime64[ns] 2020-09-01 ... 2020-09-30T23:00:00
Data variables:
    u100       (time, expver, latitude, longitude) float32 -1.4370654 ... nan
    v100       (time, expver, latitude, longitude) float32 -2.3845506 ... nan
    u10n       (time, expver, latitude, longitude) float32 -0.5837817 ... nan
    v10n       (time, expver, latitude, longitude) float32 -0.1704691 ... nan
    u10        (time, expver, latitude, longitude) float32 -2.1044703 ... nan
    v10        (time, expver, latitude, longitude) float32 -0.61489904 ... nan
    t2m        (time, expver, latitude, longitude) float32 292.8601 ... nan
    blh        (time, expver, latitude, longitude) float32 39.739746 ... nan
    cin        (time, expver, latitude, longitude) float32 nan nan ... nan nan
    mtpr       (time, expver, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-04 11:24:53 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [121]:
ds = xr.open_zarr(fsspec.get_mapper('gcs://em-gs-1/era5'),
                 #onsolidated=True,
                  chunks='auto')

ds

<xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 121, time: 93504)
Coordinates:
  * latitude   (latitude) float32 38.0 37.75 37.5 37.25 ... 8.75 8.5 8.25 8.0
  * longitude  (longitude) float32 68.0 68.25 68.5 68.75 ... 97.5 97.75 98.0
  * time       (time) datetime64[ns] 2010-01-01 ... 2020-08-31T23:00:00
Data variables:
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    cin        (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    mtpr       (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    u10n       (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
    v10n       (time, latitude, longitude) float32 dask.array<chunksize=(186, 31, 61), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-04 11:10:04 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [128]:
ds['t2m'].sel(latitude=slice(18,8)).mean(['latitude','longitude']).plot()

In [119]:
c = cdsapi.Client()

res = c.retrieve("reanalysis-era5-single-levels",
  {
    'product_type': 'reanalysis',
    'variable': 'boundary_layer_height',
    'year': years,
    'month': months,
    'day':  days,
    'time': times,
    'format': 'netcdf',
    'area'          : [38, 68, 8, 98],
    'grid'          : [0.25, 0.25]
  },
  '/home/jovyan/to_upload_to_gcs/temp_2.nc'
                )
print(res)

2020-11-04 11:40:26,269 INFO Welcome to the CDS
2020-11-04 11:40:26,270 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-11-04 11:40:26,448 INFO Request is queued
2020-11-04 11:40:29,163 INFO Request is running
2020-11-04 11:41:42,345 INFO Request is completed
2020-11-04 11:41:42,347 INFO Downloading http://136.156.133.41/cache-compute-0013/cache/data6/adaptor.mars.internal-1604490028.2366061-13049-7-f5db65c9-3776-492f-98fa-fd397abad7e5.nc to /home/jovyan/to_upload_to_gcs/temp_2.nc (20.1M)
2020-11-04 11:41:44,365 INFO Download rate 10M/s    


Result(content_length=21087956,content_type=application/x-netcdf,location=http://136.156.133.41/cache-compute-0013/cache/data6/adaptor.mars.internal-1604490028.2366061-13049-7-f5db65c9-3776-492f-98fa-fd397abad7e5.nc)


In [120]:
data = xr.load_dataset('/home/jovyan/to_upload_to_gcs/temp_2.nc')
# data.to_zarr(store=gcsmap, mode='a', append_dim='time')
data

<xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 121, time: 720)
Coordinates:
  * longitude  (longitude) float32 68.0 68.25 68.5 68.75 ... 97.5 97.75 98.0
  * latitude   (latitude) float32 38.0 37.75 37.5 37.25 ... 8.75 8.5 8.25 8.0
  * time       (time) datetime64[ns] 2020-09-01 ... 2020-09-30T23:00:00
Data variables:
    blh        (time, latitude, longitude) float32 39.739746 ... 673.6577
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-04 11:41:28 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...